In [1]:
import numpy as np
import pandas as pd

from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler

In [4]:
def scale_csv(csv_file_path):
    df_test = pd.read_csv(csv_file_path, index_col=0)
    
    #index가 정렬되어있지 않아서- 다시 reset후 드랍
    df_test.reset_index(inplace=True)
    df_test.drop(['index'], inplace=True, axis=1)
    
    #IBI가 null이 아닌 값만 추출
    df_IBI = df_test['IBI'].dropna()
    
    df_CT = df_test['Current Time']
    
    #필요없는 컬럼 삭제 및 원핫인코딩 부분 따로 추출
    df_test.drop(['Current Time', 'Segment ID', 'Emotion', 'Valance', 'Arousal'],inplace=True, axis=1)
    df_encoded = pd.get_dummies(df_test['Status'])
    df_test.drop("Status", inplace=True, axis=1)
    
    #정규화 과정(원래 데이터 프레임 + IBI 정규화)
    scaler = MinMaxScaler()
    df_test_scaled = scaler.fit_transform(df_test)
    df_test_scaled = pd.DataFrame(df_test_scaled, columns=df_test.columns)
    
    df_IBI_scaled = scaler.fit_transform(df_IBI.values.reshape(-1, 1))
    df_IBI_scaled = pd.DataFrame(df_IBI_scaled, columns=['IBI'], index=df_IBI.index)
    
    #IBI의 인덱스 추출한 부분에 다시 넣기
    df_test_scaled.loc[df_IBI.index, 'IBI'] = df_IBI_scaled.values.flatten()
    
    # df_test와 df_encoded 합치기
    new_df = pd.concat([df_test_scaled, df_encoded], axis=1)
    new_df = pd.concat([new_df, df_CT], axis=1)
    
    # 'IBI'컬럼을 마지막으로 이동
    col = list(new_df.columns)
    col.remove('IBI')
    last_col = new_df.pop('IBI')
    new_df.insert(len(col), 'IBI', last_col)
    new_df.columns = col + ['IBI']
    
    #인덱스를 datetime값으로 변경
    new_df['Current Time'] = pd.to_datetime(new_df['Current Time'], format='%Y-%m%d-%H%M-%S-%f')
    
    # 새로운 csv 파일로 저장
    new_file_path = os.path.join(os.path.dirname(os.path.dirname(csv_file_path)), f"{os.path.splitext(os.path.basename(csv_file_path))[0]}_scaled.csv")
    new_df.to_csv(new_file_path)
    
    return new_df

## 